In [1]:
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("thecrowsnestapp-creds.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
class Sailor:
    def __init__(self, name, teams, pos, rank=0, rating=1500):
        self.name = name
        self.teams = teams
        self.pos = pos
        self.rank = rank
        self.races = []
        self.changes = []
        self.rating = rating
        #add seaons
        

    def getExpScore(self,R_B):
        return 1 / (1 + np.pow(((R_B - self.rating) / 400),10))
    
    def getRatingChange(self,outcome,opponentRating):
        K = 64
        return int(K*(outcome-self.getExpScore(opponentRating)))
    
    def getNewRating(self,score,places,opponentRatings, multiplier):
        # outcomes = [0 if score > opponentPlace else 1 for opponentPlace in places]
        # ratingChanges = []
        
        # for i in range(len(outcomes)):
        #     opponentRating = opponentRatings[i]
        #     outcome = outcomes[i]
        #     ratingChanges.append(self.getRatingChange(outcome,opponentRating))
        
        # totalChange = 0
        
        # for i in range(len(ratingChanges)):
        #     change = ratingChanges[i]
        #     place = places[i]
        #     totalChange += change * (abs(place - score) / 8)
        
        # self.rating += totalChange * multiplier
        
        return np.sum([self.getRatingChange(0 if score < o else 1 ,Or) * (abs(o - score) / 8) for o,Or in zip(places, opponentRatings)]) * multiplier



In [3]:
elliott = Sailor("Elliott", 'MIT','Crew')

# elliott.getNewRating(1,[2,3,4],[2000,2000,2000], 1)
print(elliott.getNewRating(1,[2],[1100], 1))
elliott.rating = 1100
print(elliott.getNewRating(2,[1],[1500], 1))

-4.0
4.0


In [87]:
df_races = pd.read_csv("races.csv",converters={"Teams": lambda x: [y.strip().split("'")[1] for y in x.strip("[]").split(", ")]})

df_races['Ratio'] = 1 - (df_races['Score'] / df_races['Teams'].apply(len))
df_races['Ratio'] = df_races['Ratio'].astype(float)
df_races['numTeams'] = df_races['Teams'].apply(len)
df_races['raceNum'] = df_races['raceID'].apply(lambda id: int(id.split("/")[2][:-1]))

df_races['Date'] = df_races['Date'].apply(lambda date: (int(date.split("-")[0]), int(date.split("-")[1]), int(date.split("-")[2])))
df_races = df_races.sort_values(['Date', 'raceNum']).reset_index(drop=True)

# df_races = df_races.loc[df_races['raceID'].str.contains('f24')] # filter for f24
df_races_skipper = df_races.loc[df_races['Position'].str.contains('Skipper')] # filter for skippers
df_races_crew = df_races.loc[df_races['Position'].str.contains('Crew')] # filter for skippers

people = {}
for p in df_races_skipper['Sailor'].unique():
    # pos = df_races.loc[df_races['Sailor'] == p]['Position'].iat[0] if len(df_races.loc[df_races['Sailor'] == p]['Position']) > 0 else "Unknown"
    teams = df_races_skipper.loc[df_races['Sailor'] == p]['Team'].unique() if len(df_races_skipper.loc[df_races_skipper['Sailor'] == p]['Team']) > 0 else "Unknown"
    people[p + "/" + 'Skipper'] = Sailor(p, teams, 'Skipper')
for p in df_races_crew['Sailor'].unique():
    # pos = df_races.loc[df_races['Sailor'] == p]['Position'].iat[0] if len(df_races.loc[df_races['Sailor'] == p]['Position']) > 0 else "Unknown"
    teams = df_races_crew.loc[df_races['Sailor'] == p]['Team'].unique() if len(df_races_crew.loc[df_races_crew['Sailor'] == p]['Team']) > 0 else "Unknown"
    people[p + "/" + 'Crew'] = Sailor(p, teams, 'Crew')

In [88]:
people

{'Bastien Rasse/Skipper': <__main__.Sailor at 0x31b3863f0>,
 'Samuel Patton/Skipper': <__main__.Sailor at 0x15e9dc590>,
 'Alex Bowdler/Skipper': <__main__.Sailor at 0x15e9dcf80>,
 'Peter Cronin/Skipper': <__main__.Sailor at 0x15e9dc560>,
 'Joseph Hou/Skipper': <__main__.Sailor at 0x15e9defc0>,
 'Edward Ansart/Skipper': <__main__.Sailor at 0x15e9dc3e0>,
 'Pearl Lattanzi/Skipper': <__main__.Sailor at 0x15e9dd970>,
 'Molly Hanrahan/Skipper': <__main__.Sailor at 0x15e9dd940>,
 'Mercy Tangredi/Skipper': <__main__.Sailor at 0x15e9dc380>,
 'Morgan Carew/Skipper': <__main__.Sailor at 0x15e9dd8e0>,
 'Trey Summers/Skipper': <__main__.Sailor at 0x15e937aa0>,
 'Theodore Knobel/Skipper': <__main__.Sailor at 0x15e936ea0>,
 'Hayden Lahr/Skipper': <__main__.Sailor at 0x15e936090>,
 'Frances Tarpey-Schwed/Skipper': <__main__.Sailor at 0x15e935f40>,
 'Thomas Pentimonti/Skipper': <__main__.Sailor at 0x15e937bc0>,
 'Emily Smith/Skipper': <__main__.Sailor at 0x15e92db50>,
 'Lucas Burzycki/Skipper': <__main

In [90]:
stype = 'Ratio'
i = 0
for type, df_races in zip(['/Skipper', '/Crew'],[df_races_skipper, df_races_crew]):
    for date in list(df_races['Date'].unique()):
        for regatta in list(df_races.loc[df_races['Date'] == date,'Regatta'].unique()):
            for race in df_races.loc[(df_races['Regatta'] == regatta) ,'raceID'].unique():
                if i % 100 == 0:
                    print(f"Currently analyzing race {i}/{len(df_races['raceID'].unique())} Regatta:{regatta}, Date:{date}")
                i += 1
                
                scores = df_races.loc[df_races['raceID'] == race]

                sailors = scores['Sailor']
                if len(sailors) == 0:
                    continue
                
                globalAvg = sum([p.rating for p in people.values()]) / len(people.keys())
                regattaAvg = sum([people[p + type].rating for p in sailors]) / len(sailors)
                multiplier = regattaAvg / globalAvg

                for sailor, actual_score in zip([people[s + type] for s in sailors], df_races[stype]):
                    sailorScore = scores.loc[scores['Sailor'] == sailor.name][stype].iat[0]
                    
                    partner = scores.loc[scores['Sailor'] == sailor.name, 'Partner'].iat[0]
                    
                    opponentScores = [scores.loc[scores['Sailor'] == p, stype].iat[0] for p in sailors if p != sailor.name and p != partner ]
                    opponentRatings = [people[p + type].rating for p in sailors if p != sailor.name and p != partner]
                    
                    change = sailor.getNewRating(sailorScore, opponentScores, opponentRatings, multiplier)
                    
                    sailor.rating += change
                    
                    sailor.races.append(race)
                    sailor.changes.append((scores.loc[scores['Sailor'] == sailor.name]['Score'].iat[0],change, regattaAvg, sailor.rating, date, partner, scores.loc[scores['Sailor'] == sailor.name]['Ratio'].iat[0]))
                    

Currently analyzing race 0/12708 Regatta:s22/andrew-johnson, Date:(2022, 1, 15)
Currently analyzing race 100/12708 Regatta:s22/bryson-women-fleet-race, Date:(2022, 2, 20)
Currently analyzing race 200/12708 Regatta:s22/odu-women, Date:(2022, 3, 5)
Currently analyzing race 300/12708 Regatta:s22/saisa-south-points-2022, Date:(2022, 3, 13)
Currently analyzing race 400/12708 Regatta:s22/connecticut-valley-dinghy, Date:(2022, 3, 26)
Currently analyzing race 500/12708 Regatta:s22/central-series-march-27, Date:(2022, 3, 27)
Currently analyzing race 600/12708 Regatta:s22/north-designate-st-francis, Date:(2022, 4, 2)
Currently analyzing race 700/12708 Regatta:s22/mcsa-open-fleet-race, Date:(2022, 4, 9)
Currently analyzing race 800/12708 Regatta:s22/south-fleet-race-ewenson, Date:(2022, 4, 9)
Currently analyzing race 900/12708 Regatta:s22/maisa-spring-women, Date:(2022, 4, 16)
Currently analyzing race 1000/12708 Regatta:s22/spring-oberg, Date:(2022, 4, 16)
Currently analyzing race 1100/12708 Rega

In [ ]:
col = db.collection('sailorsElo')
batch = db.batch()

for i,p in enumerate(people.values()):
    batch.set(col.document(),
        {"Name":p.name, 
         "Position": p.pos,
         "Teams": list(p.teams),
         "Rating": p.rating,
         "GlobalRank": p.rank,
         "races": [{"raceID": raceid, 
                    "score": float(c[0]), 
                    "change": float(c[1]), 
                    'regAvg': float(c[2]), 
                    'newRating': float(c[3]),
                    'date': c[4],
                    'partner':c[5],
                    'ratio': float(c[6])
                    } for raceid, c in zip(p.races, p.changes)]})
    if i % 20 == 0: # commit every 20 documents
        batch.commit()
batch.commit()

wrote to db 88
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to db 472
wrote to 

In [98]:
df_elo = pd.DataFrame(columns=['Sailor', 'Teams', 'Pos', 'Elo'])
# pdict = {}
new_rows = []
for p in people.values():
    df_elo.loc[len(df_elo)] = [p.name, p.teams, p.pos, p.rating]
    # person_dict = dict(zip(p.races, p.changes))
    # person_dict["Sailor"] = p.name
    row = {"Sailor": p.name}
    row.update(zip(p.races, [(int(c[0]), float(c[1]), float(c[2]), float(c[3])) for c in p.changes]))
    new_rows.append(row)


# new_df = pd.DataFrame.from_dict(pdict, orient='index').reset_index()
# new_df.rename(columns={'index': 'Sailor'}, inplace=True)
# new_df = pd.DataFrame(new_rows)

# all_keys = ['Sailor'] + list({race for p in people.values() for race in p.races})
all_keys = ['Sailor'] + list(df_races['raceID'].unique())
new_df = pd.DataFrame(new_rows, columns=all_keys)

df_elo = pd.concat([df_elo.set_index('Sailor'), new_df.set_index('Sailor')], axis=1).reset_index()

df_elo = df_elo.sort_values('Elo', ascending=False)
df_elo.reset_index(drop=True, inplace=True)

for p in list(people.values())[:5]:
    p.rank = df_elo.loc[df_elo['Sailor'] == p.name].index[0]

# df_elo.to_csv("elo17.csv", index=False)

In [39]:
f24_sailors = df_races.loc[df_races['raceID'].str.contains('f24'),'Sailor'].unique()

df_elof24 = df_elo.loc[df_elo["Sailor"].isin(f24_sailors)].reset_index(drop=True)

df_sailor = df_elof24[df_elof24['Sailor'] == "Carter Anderson"]
df_sailor = df_sailor.dropna(axis=1, how='all')

df_sailor

,Sailor,Team,Pos,Elo,f23/stoney-burke-co-ed-varsity/1A,f23/stoney-burke-co-ed-varsity/2A,f23/stoney-burke-co-ed-varsity/5A,f23/stoney-burke-co-ed-varsity/6A,f23/stoney-burke-co-ed-varsity/9A,f23/stoney-burke-co-ed-varsity/10A,...,f24/oberg/5B,f24/oberg/6B,f24/oberg/7B,f24/oberg/8B,f24/terrier-invite/1A,f24/terrier-invite/2A,f24/terrier-invite/3A,f24/terrier-invite/4A,f24/terrier-invite/5A,f24/terrier-invite/6A
144,Carter Anderson,UC Santa Cruz,Skipper,2727.272097,"(3, 11.215216292446081, 1366.755737738248, 151...","(4, 10.03804306945171, 1365.4095084842568, 152...","(8, 3.696158060878756, 1345.6326944371062, 152...","(9, -0.35171337383966694, 1346.2994337701496, ...","(10, -12.446481258868403, 1376.9471782160645, ...","(3, 14.109478671946865, 1379.0650288283075, 15...",...,"(14, 3.0274001216684234, 1981.2545942518332, 2...","(10, 14.459560203696054, 1988.7388794561746, 2...","(15, 1.35512743731108, 2037.7470309641953, 266...","(10, 11.399537001676835, 2094.8752033629676, 2...","(17, -5.0414302083236135, 1389.7162119788536, ...","(16, -4.31673943954472, 1390.3624457297137, 26...","(5, 24.75931469327344, 1388.917150848889, 2687...","(9, 7.86529557367321, 1385.9644202247694, 2694...","(8, 16.757303736423253, 1384.3177220215214, 27...","(6, 15.586821992695935, 1443.2715047482573, 27..."


In [ ]:
import requests
from bs4 import BeautifulSoup

batch = db.batch()
col = db.collection('eloTeams')
lenteams = len(list(df_races['Team'].unique()))
teams = []

print(lenteams)
for i,team in enumerate(list(df_races['Team'].unique())):
    print(f"{i}/{lenteams} {team}")
    avg = df_elo.loc[df_elo['Teams'].apply(lambda x: team in x), 'Elo'].mean()
    region = df_races
    
    teamLink = df_races.loc[df_races['Team'] == team, 'Teamlink'].iloc[0]
    url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"
    # print(url)
    page = requests.get(url)
    teamPage = BeautifulSoup(page.content, 'html.parser')
    
    try:
        region = teamPage.find('span', class_="page-info-value").contents[0].contents[0]
        # print(team, avg, region)
    except:
        print(url)
        
    members = [{"name": p.name, 'pos':p.pos, 'rating': p.rating, 'globalrank': p.rank} for p in people.values() if team in p.teams]
    
    teams.append({"name":team, "avg": avg,"region": region, "link": url})
    batch.set(col.document(),{"name":team, "avg": avg,"region": region, "link": url, 'members': members})
    if i % 20 == 0: # commit every 20 documents
            batch.commit()
batch.commit()
doc = db.collection('vars').document('eloTeams').set({"teams": teams})

177
0/177 Hawaii
1/177 Salve Regina
2/177 Southern Cal
3/177 Washington
4/177 Western Washington
5/177 UC Los Angeles
6/177 UC San Diego
7/177 Cal Poly
8/177 UC Davis
9/177 Virginia
10/177 Berkeley
11/177 Cal Maritime
12/177 San Diego State
13/177 UC Santa Cruz
14/177 Arizona State
15/177 UC Irvine
16/177 UC Santa Barbara
17/177 Texas
18/177 North Texas
19/177 Texas A&M Galveston
20/177 Georgia Tech
21/177 U. Miami
22/177 North Carolina State
23/177 Florida State
24/177 Eckerd
25/177 Jacksonville
26/177 U South Carolina
27/177 South Florida
28/177 Citadel
29/177 Rollins
30/177 Duke
31/177 Florida Tech
32/177 Florida
33/177 Embry-Riddle
34/177 UNC Wilmington
35/177 Auburn
36/177 Stanford
37/177 Charleston
38/177 Princeton
39/177 Christopher Newport
40/177 Hampton
41/177 Northwestern
42/177 George Washington
43/177 Coast Guard
44/177 St. Mary's
45/177 Navy
46/177 Old Dominion
47/177 Clemson
48/177 Georgia
49/177 Vanderbilt
50/177 Brown
51/177 Roger Williams
52/177 Connecticut College
53/